<a href="https://colab.research.google.com/github/gvigkannan/Model_ADay/blob/PyTorch/Zero2GANS_MNIST_Pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torchvision
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
from torchvision.datasets import MNIST
from torchvision.transforms import ToTensor
from torchvision.utils import make_grid
from torch.utils.data.dataloader import DataLoader
from torch.utils.data import random_split

In [ ]:
%matplotlib inline
plt.rcParams['figure.facecolor'] = '#ffffff'

## Dataset Processing

In [ ]:
dataset = MNIST(root = 'data/',
                download = True,
                transform = ToTensor())

In [ ]:
np.shape(dataset[0][0])

torch.Size([1, 28, 28])

In [ ]:
# Data Prep
val_size = int(len(dataset) * 0.1)
train_size = len(dataset) - val_size
train_ds, val_ds = random_split(dataset, 
                                [train_size, val_size])
len(train_ds), len(val_ds)

(54000, 6000)

In [ ]:
batch_size = 128 # Hyperparameter!
train_loader = DataLoader(train_ds,
                          batch_size,
                          shuffle = True,
                          num_workers = 4,
                          pin_memory = True)
val_loader = DataLoader(val_ds,
                        batch_size * 2,
                        num_workers = 4,
                        pin_memory = True)

## Model

In [ ]:
def accuracy(outputs, labels):
    _, preds = torch.max(outputs, dim=1)
    return torch.tensor(torch.sum(preds == labels).item() / len(preds))

In [ ]:
class MNIST_Model(nn.Module):
    """
    Feedforward NN with 1 Hidden Layer
    """
    def __init__(self, in_size, hidden_size, out_size):
        super().__init__()
        self.linear_1 = nn.Linear(in_size, hidden_size)
        self.linear_2 = nn.Linear(hidden_size, out_size)

    def forward(self, xb):
        xb = xb.view(xb.size(0), -1)
        out = self.linear_1(xb)
        out = F.relu(out)

        out = self.linear_2(out)
        return out
    
    def training_step(self, batch):
        images, labels = batch
        out = self(images)
        loss = F.cross_entropy(out, labels)
        return loss

    def validation_step(self, batch):
        images, labels = batch
        out = self(images)
        loss = F.cross_entropy(out, labels)
        acc = accuracy(out, labels)
        return {'val_loss': loss, 'val_acc': acc}

    def validation_epoch_end(self, outputs):
        batch_losses = [x['val_loss'] for x in outputs]
        epoch_loss = torch.stack(batch_losses).mean()

        batch_accs = [x['val_acc'] for x in outputs]
        epoch_acc = torch.stack(batch_accs).mean()
        return {'val_loss': epoch_loss.item(),
                'val_acc': epoch_acc.item()}

    def epoch_end(self, epoch, result):
        print("Epoch [{}], val_loss: {:.4f}, val_acc: {:.4f}".format(epoch,
                                                                    result['val_loss'],
                                                                    result['val_acc']))
        

In [ ]:
input_size = 784
hidden_size = 32
num_classes = 10

In [ ]:
model_temp = MNIST_Model(input_size, hidden_size,
                         num_classes)

In [ ]:
for t in model_temp.parameters():
    print(t.shape)


torch.Size([32, 784])
torch.Size([32])
torch.Size([10, 32])
torch.Size([10])


In [ ]:
# Trial Test!
for images, labels in train_loader:
    outputs = model_temp(images)
    loss = F.cross_entropy(outputs, labels)
    print('Loss', loss.item())
    break

Loss 2.304720878601074


In [ ]:
print("Model Output Shape: {}".format(outputs.shape))
print("Model Output Sample: \n",outputs[:2].data)



Model Output Shape: torch.Size([128, 10])
Model Output Sample: 
 tensor([[-0.1247,  0.1444,  0.0148, -0.2210, -0.1035,  0.0022,  0.0032, -0.0825,
         -0.1443,  0.0598],
        [-0.1617,  0.1580,  0.0629, -0.2119, -0.0756, -0.0991,  0.1838, -0.1155,
         -0.1077,  0.0363]])


## GPU Related!


In [ ]:
def get_default_device():
    """ Pick GPU if available, else CPU"""
    if torch.cuda.is_available():
         return torch.device('cuda')
    else:
        return torch.device('cpu')
        

In [ ]:
device = get_default_device()
device

device(type='cpu')

In [ ]:
def to_device(data, device):
    """ Move tensor(s) to chosen device!
    """

    if isinstance(data, (list, tuple)):
        return [to_device(x, device) for x in data]
    return data.to(device, non_blocking = True)

In [ ]:
for images, labels in train_loader:
    print(images.shape)
    images = to_device(images, device)
    print(images.device)
    break

torch.Size([128, 1, 28, 28])
cpu


In [ ]:
class DeviceDataLoader():
    """ Wrap a dataloader to move data to device"""
    def __init__(self, dl, device):
        self.dl = dl 
        self.device = device

    def __iter__(self):
        """ Yield a batch of data after moving it a device!"""
        for b in self.dl:
            yield to_device(b, self.device)

    def __len__(self):
        """ Number of batches """
        return len(self.dl)

## Training the Model!

In [ ]:
def evaluate(model, val_loader):
    """ Evaluate the model's performance on the validation set """
    outputs = [model.validation_step(batch) for batch in val_loader]
    return model.validation_epoch_end(outputs)

def fit(epochs, lr, model, train_loader, val_loader, opt_func = torch.optim.SGD):
    """ Train the model using SGD"""
    history = []
    optimizer = opt_func(model.parameters(), lr)

    for epoch in range(epochs):
        for batch in train_loader:
            loss = model.training_step(batch)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()

        result = evaluate(model, val_loader)
        model.epoch_end(epoch, result)
        history.append(result)
    return history


In [ ]:
model = MNIST_Model(input_size, hidden_size, num_classes)
to_device(model, device)

MNIST_Model(
  (linear_1): Linear(in_features=784, out_features=32, bias=True)
  (linear_2): Linear(in_features=32, out_features=10, bias=True)
)

In [ ]:
history = [evaluate(model, val_loader)]

In [ ]:
history += fit(5, 0.5, model, train_loader, val_loader)

Epoch [0], val_loss: 0.2111, val_acc: 0.9396
Epoch [1], val_loss: 0.1754, val_acc: 0.9476
Epoch [2], val_loss: 0.1512, val_acc: 0.9575
Epoch [3], val_loss: 0.1340, val_acc: 0.9615
Epoch [4], val_loss: 0.1321, val_acc: 0.9603


In [ ]:
history += fit(5, 0.1, model, train_loader, val_loader)

Epoch [0], val_loss: 0.1160, val_acc: 0.9654
Epoch [1], val_loss: 0.1160, val_acc: 0.9654
Epoch [2], val_loss: 0.1138, val_acc: 0.9663
Epoch [3], val_loss: 0.1130, val_acc: 0.9665
Epoch [4], val_loss: 0.1171, val_acc: 0.9643


In [ ]:
history += fit(5, 0.01, model, train_loader, val_loader)

Epoch [0], val_loss: 0.1120, val_acc: 0.9655
Epoch [1], val_loss: 0.1114, val_acc: 0.9658
Epoch [2], val_loss: 0.1115, val_acc: 0.9656
Epoch [3], val_loss: 0.1114, val_acc: 0.9656
Epoch [4], val_loss: 0.1113, val_acc: 0.9660


## Testing

In [ ]:
test_dataset = MNIST(root='data/', 
                     train=False,
                     transform=ToTensor())

In [ ]:
test_loader = DeviceDataLoader(DataLoader(test_dataset, batch_size=256), device)
result = evaluate(model, test_loader)
result

{'val_acc': 0.9688476324081421, 'val_loss': 0.10450003296136856}